# Setup

In [ ]:
#Check directory
#!pwd

In [ ]:
#Change directory
#%cd /Users/georgesmith/Desktop/RTP/DONOR DATA ANALYSIS

In [ ]:
#Import necessary packages:
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import datetime
from datetime import date
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from IPython.display import display
#import sys
#Install pingouin package into this jupyter kernal
#!{sys.executable} -m pip install pingouin
#import pingouin as pg #this throws the warning message
#!{sys.executable} -m pip install --upgrade statsmodels
#import pickle
import plotly.offline as py
color = sns.color_palette()
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import plotly.tools as tls


%matplotlib inline

In [ ]:
#Read in donation data:
input_cols = ['donation_id',
 'donation_name',
 'donation_record_type',
 'amount',
 'close_date',
 'account_name',
 'account_id',
 'account_record_type',
 'contact_id',
 'pledge_amount',
 'fund',
 'payment_type',
 'fiscal_quarter',
 'fiscal_year',
 'type',
 'frequency',
 'donation_matched',
 'amount_matched',
 'matching_close_date',
 'first_donation_date',
 'last_donation_amount',
 'last_donation_date',
 'lifetime_donation_count',
 'lifetime_donation_total_amount',
 'lifetime_order_count',
 'lifetime_single_ticket_purchase_total',
 'lifetime_subscription_purchase_total',
 'do_not_mail',
 'email_opt_out',
 'account_created_date',
 'contact_email',
 'payment_schedule',
 'stage',
 'contact_phone',
 'account_phone',
 'amount_due',
 'amount_paid',
 'anonymous',
 'card_type',
 'dedication_type',
 'donation_origin',
 'donor_name',
 'donor_last_first',
 'amount_donated_last_365_days',
 'amount_donated_last_fiscal_year',
 'amount_donated_this_fiscal_year',
 'description',
 'group_sales_total',
 'ticket_order_id',
 'total_amount',
 'transaction_id',
 'last_transaction_id',
 'order_count_last_#_days',
 'mailing_street',
 'mailing_city',
 'mailing_state',
 'mailing_zip']
raw_donations = pd.read_csv('/Users/georgesmith/Desktop/RTP/DONOR DATA ANALYSIS/donations_by_acct_contact_July_2014_Aug_06_2020.csv', header=0, names=input_cols)

In [ ]:
#Remove columns necessary to anonymize each record
iden_cols = ['donation_name', 'account_name', 'donation_matched', 'contact_email', 'contact_phone', 'account_phone', 'donor_name', 'donor_last_first', 'mailing_street']
donations = raw_donations.drop(iden_cols, axis=1, inplace=False)
donations.head()

In [ ]:
donations.shape

In [ ]:
#Get variable dataypes and non-null counts:
donations.info()

In [ ]:
def col_dtype_conversions(df, col_list1, new_dtype1, col_list2, new_dtype2):
    """
    Converts columns in col_list1 to datatype specified in new_dtype1
    and converts those in col_list2 to datatype specified in new_dtype2

    Params
    ------
    df : pandas.DataFrame
        dataframe with columns in both col_lists
    col_list1 : list
        list of column names to be converted to dtype specified by new_dtype1
    new_dtype1 : str (must put in quotes)
        data type to which columns in col_list1 will be converted
    col_list2 : list
        list of column names to be converted to dtype specified by new_dtype2
    new_dtype2 : str (must put in quotes)
        data type to which columns in col_list2 will be converted

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns, some of which are now cast as
        the specified data types
    """
    for col1 in col_list1:
        df[col1] = df[col1].astype(new_dtype1)
        for col2 in col_list2:
            df[col2] = df[col2].astype(new_dtype2)
    return

In [ ]:
col1 = ['fiscal_year', 'transaction_id']
col2 = ['close_date', 'first_donation_date', 'last_donation_date', 'account_created_date']

In [ ]:
col_dtype_conversions(donations, col1, 'str', col2, 'datetime64[ns]')

In [ ]:
#Confirm dtype conversions
donations.dtypes

In [ ]:
#Remove all refunded donation line items to omit them from analysis and calculations
donations = donations[donations['stage'] != 'Refunded']

In [ ]:
#Show descriptive statistics for non-numeric columns in donations df
#Top - most common value in given column; freq - frequency of column's most common value
donations.describe(include=['O']).T

In [ ]:
#Show descriptive statistics for numeric columns in donations df
donations.describe().T

In [ ]:
#Remove columns that are already clearly extraneous and/or lacking variability
donations.drop(['matching_close_date'], axis=1, inplace=True)

In [ ]:
#Isolate the mean donation amount(for use later in outlier identification)
donations.describe().T.at['amount', 'mean']

In [ ]:
#See if rows where amount == 0 have other valuable info
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
donations.loc[donations['amount'] == 0]

Records where amount is 0 appear to lack other necessary info so we'll remove them. This will also enhance accuracy of median imputation below.

In [ ]:
#Remove rows where amount == 0
donations = donations.loc[donations['amount'] != 0]

# Initial research question: are donations (volume and dollar total) increasing overtime?

**Handle Null Values**

In [ ]:
#Which columns have nan records (and how many)?
num_nan = donations.isna().sum()
print(num_nan[num_nan >0])

Amount:

In [ ]:
#Review the records to understand how to handle these nulls
donations.loc[donations['amount'].isna()].reset_index(drop=True)

In [ ]:
dons_to_research = raw_donations.loc[(raw_donations['account_id'].isin(['0013600000VKRpL', '0013600000VKQmI'])) & (raw_donations['amount'].isna()) & (raw_donations['donation_record_type'] == 'Pledge Payment')]

**The null pledge payment amounts for the account ids referenced in the cell above seem to be the result of a failure to record the amounts for which those donation checks were issued. Research these amounts and update the CRM accordingly.**

In [ ]:
#Separate pledge records from actual donation records
pledges = donations.loc[donations['donation_record_type'] == 'Pledge', 
                        ['donation_id', 'donation_record_type', 'amount',
       'close_date', 'account_id', 'account_record_type', 'contact_id', 'pledge_amount', 'fund', 
       'fiscal_year', 'payment_schedule', 'type', 'first_donation_date', 'last_donation_amount',
       'last_donation_date', 'lifetime_donation_count',
       'lifetime_donation_total_amount', 'lifetime_order_count',
       'lifetime_single_ticket_purchase_total',
       'lifetime_subscription_purchase_total', 'account_created_date', 'amount_donated_last_365_days',
       'amount_donated_last_fiscal_year', 'amount_donated_this_fiscal_year',
       'ticket_order_id', 'order_count_last_#_days',
       'mailing_city', 'mailing_state', 'mailing_zip']]

In [ ]:
#Update donations df to exclude the two pledge payments with nan amounts and all pledge records (as these are not actually donations)
donations = donations.loc[~donations['amount'].isna()]
#Confirm pledge rows are eliminated
donations.donation_record_type.value_counts(dropna=False)

In [ ]:
#Drop Pledge Amount column given pledge rows have been removed
donations.drop(['pledge_amount'], axis=1, inplace=True)

In [ ]:
#Check to ensure the amount column in all rows is now populated
donations.amount.isna().sum()

In [ ]:
#NAMED TUPLE EXAMPLE
# from collections import namedtuple
# >>> Student = namedtuple("Student", "name age student_id")
# >>> student = Student("John Smith", 17, 178044)
# >>> print(f"name: {student.name}\nage: {student.age}\nstudent id: {student.student_id}")
# name: John Smith
# age: 17
# student id: 178044

In [ ]:
#Recheck amount stats
donations.describe().T

**Visualize input frequency distributions for continuous features**

In [ ]:
col_names = ['amount',
 'amount_matched',
 'last_donation_amount',
 'lifetime_donation_count',
 'lifetime_donation_total_amount',
 'lifetime_order_count',
 'lifetime_single_ticket_purchase_total',
 'lifetime_subscription_purchase_total',
 'amount_due',
 'amount_paid',
 'amount_donated_last_365_days',
 'amount_donated_last_fiscal_year',
 'amount_donated_this_fiscal_year',
 'group_sales_total',
 'total_amount',
 'order_count_last_#_days']

fig, ax = plt.subplots(len(col_names), figsize=(16,12))

for i, col_val in enumerate(col_names):

    sns.distplot(donations[col_val], hist=True, ax=ax[i])
    ax[i].set_title('Freq dist '+col_val, fontsize=10)
    ax[i].set_xlabel(col_val, fontsize=8)
    ax[i].set_ylabel('Count', fontsize=8)

plt.show()

In [ ]:
figure, ax = plt.subplots(5,2, figsize=(12,24))

#See the distrubution of the data
sns.distplot(donations['amount'],ax= ax[0,0])
sns.distplot(donations['last_donation_amount'],ax=ax[0,1])
sns.distplot(donations['lifetime_donation_count'],ax= ax[1,0])
sns.distplot(donations['lifetime_donation_total_amount'],ax= ax[1,1])
sns.distplot(donations['lifetime_single_ticket_purchase_total'],ax= ax[2,0])
sns.distplot(donations['lifetime_subscription_purchase_total'],ax=ax[2,1])
sns.distplot(donations['amount_donated_last_365_days'],ax= ax[3,0])
sns.distplot(donations['amount_donated_last_fiscal_year'],ax= ax[3,1])
sns.distplot(donations['lifetime_order_count'],ax= ax[4,0])
sns.distplot(donations['order_count_last_#_days'],ax= ax[4,1])

plt.show() #optional line -- eliminates strange text output before plots

last_donation_amount:

In [ ]:
#Which records have null last donation amounts?
donations.loc[donations.last_donation_amount.isna()]

In [ ]:
#last_donation_amount seems to be null for records that represent an account's first donation. 
#As such, impute NaN with 0 when close_date == first_donation_date
donations['last_donation_amount'] = np.where(((donations['last_donation_amount'].isna()) & (donations['close_date'] == donations['first_donation_date'])), 0, donations['last_donation_amount'])

In [ ]:
#Recheck which records have null last donation amounts?
donations.loc[donations.last_donation_amount.isna()]

In [ ]:
#Looked up the account in the CRM. last_donation_amount for record above is $5,000. Impute accordingly.
donations.loc[donations.last_donation_amount.isna(), 'last_donation_amount'] = 5000

Ensure next data pull includes the record referenced in last_donation_amount above (i.e. account_id == '0013600000VKR96' & close_date == '2016-10-24')

In [ ]:
#Check result -- Which records have null last donation amounts?
donations.loc[donations.last_donation_amount.isna()]

Fund Column:

In [ ]:
#Check out non-household / non-individual records with NaN in fund column
donations.loc[((donations['fund'].isna()) | (donations['fund'].isin(["Annual Fund Historical", 'ANN']))) & (~donations['account_record_type'].isin(['Household', 'Individual']))].reset_index(drop=True)

In [ ]:
#Check out household / individual records with NaN in fund column
donations.loc[((donations['fund'].isna()) | (donations['fund'].isin(["Annual Fund Historical", 'ANN']))) & (donations['account_record_type'].isin(['Household', 'Individual']))]

In [ ]:
#Convert nan in fund column (except for pledges, which have already been filtered into pledges df) to Annual Fund.
#Also, replace other AF naming conventions with with 'Annual Fund'
donations.loc[(donations['fund'].isna()) | (donations['fund'].isin(["Annual Fund Historical", 'ANN'])), 'fund'] = 'Annual Fund'

In [ ]:
#Consolidate levels referencing the same fund under common levels
donations.loc[(donations['fund'].isin(["25th Anniversary Campaign Fund", '25th Anniversary Fund', '25TH'])), 'fund'] = "25th Anniversary Capital Campaign"
donations.loc[(donations['fund'] == 'NEXT'), 'fund'] = "Next Stage Campaign"
donations.loc[(donations['fund'] == 'John Knapp -Tim Gillham Fund'), 'fund'] = "New Works (Knapp-Gillham) Fund"
donations.loc[(donations['fund'].isin(["Education Program Fund", 'EDUC'])), 'fund'] = "Education Fund"
donations.loc[(donations['fund'] == 'MMP'), 'fund'] = "Mary Morton Parsons Challenge Grant Match"

In [ ]:
#Check fund column value changes
donations.fund.value_counts(dropna=False)

payment_type:

In [ ]:
donations.payment_type.value_counts(dropna=False)

In [ ]:
#Assign group levels referencing the same payment_type to common levels
donations.loc[(donations['payment_type'] == 'CRED'), 'payment_type'] = "Credit Card"
donations.loc[(donations['payment_type'] == 'Third Party'), 'payment_type'] = "Credit Card - Third Party"
donations.loc[(donations['payment_type'] == 'INKND'), 'payment_type'] = "In Kind"

In [ ]:
donations.loc[donations.payment_type.isna()].sort_values(by=['donation_record_type'])

In [ ]:
#Fill NaN in payment_type with Ticket Order Refund if donation_origin == Ticket Order Refund
donations.loc[(donations.payment_type.isna()) & (donations.donation_origin == 'Ticket Order Refund'), 'payment_type'] = 'Ticket Order Refund'

In [ ]:
#The remaining NaN records in the payment_type column seem to be checks (based on spot checking some records in the CRM):
donations.loc[donations.payment_type.isna(), 'payment_type'] = 'Check'

In [ ]:
donations.payment_type.value_counts(dropna=False)

In [ ]:
#Define function to show value counts of categorical vars specified in column list input
def cat_col_value_counts(df, cat_col_list):
    x=[]
    """
    Prints value counts of each categorical variable from df that is listed in cat_col_list 

    Params
    ------
    df : pandas.DataFrame
        dataframe containing columns in cat_col_list
    cat_col_list : list
        list of column names for which value counts will be printed

    Returns
    -------
    prints value counts for each variable in cat_col_list
    """

    for cat_col in cat_col_list:
        x.append(df[cat_col].value_counts(dropna= False))
        pd.Series(x).sort_index()
    print (x)

In [ ]:
#Title case the mailing_city to help join like levels
donations['mailing_city'] = donations.mailing_city.str.title()

In [ ]:
cat_col_list = ['account_record_type', 'type', 'payment_schedule', 'frequency', 'stage', 'card_type', 'donation_origin', 'mailing_city', 'mailing_state']

In [ ]:
cat_col_value_counts(donations, cat_col_list)

mailing_city:

mailing_city value_counts() output illustrates need to standardize city name storage conventions (i.e. Fredericksburg and FREDERICKSBURG are the same city, but currently stored differently)

In [ ]:
#Sort mailing city value counts by index to see which cities are the same, but written differently
donations['mailing_city'].value_counts(dropna=False).to_frame().sort_index()

In [ ]:
#Consolidate cities that are the same, but written differently
donations.loc[(donations['mailing_city'] == 'Charlottesvle'), 'mailing_city'] = "Charlottesville"
donations.loc[donations['mailing_city'].isin(['Manakin Sabot Virginia', 'Manakin-Sabot']), 'mailing_city'] = "Manakin Sabot"
donations.loc[donations['mailing_city'].isin(['Mechanicsvile', 'Mechsnicsville']), 'mailing_city'] = "Mechanicsville"
donations.loc[donations['mailing_city'].isin(['N. Chesterfield', 'N.Chesterfield', 'No Chesterfield', 'North Chestefield', 'N Chesterfield']), 'mailing_city'] = "North Chesterfield"
donations.loc[donations['mailing_city'].isin(['Richmmond', 'Rickmond']), 'mailing_city'] = 'Richmond'

#Recheck mailing_city value_counts():
donations['mailing_city'].value_counts(dropna=False).to_frame().sort_index()

In [ ]:
#Create binary flags columns to mark nulls in mailing_city and some other categorical columns:

cat_cols = ['account_record_type', 'type', 'stage', 'card_type', 'mailing_city', 'mailing_state']

for col in cat_cols:
    flag_col = col + '_null_flag'
    donations[flag_col] = np.where(donations[col].isnull(), 1, 0)

In [ ]:
donations.iloc[:, 46:].head()

In [ ]:
#Impute NaN in the following columns with "not stored" - ['donation_origin', 'card_type', 'type', 'mailing_city', 'mailing_state']
donations.loc[:, ['donation_origin', 'card_type', 'type', 'mailing_city', 'mailing_state']] = donations[['donation_origin', 'card_type', 'type', 'mailing_city', 'mailing_state']].fillna('not stored')

In [ ]:
#Drop categoricals with large proportion of random nulls
donations.drop(['payment_schedule', 'frequency', 'donation_origin', 'dedication_type', 'description', 'ticket_order_id', 'last_transaction_id', 'mailing_zip'], axis=1, inplace=True)

Create campaign column

In [ ]:
#Assign regular appeal campaigns
donations.loc[(donations['type'] == 'GIVINGTUESDAYNOW'), 'campaign'] = 'GIVINGTUESDAYNOW (S27)'
donations.loc[(donations['close_date'] > '2020-06-05') & (donations['close_date'] < '2020-08-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Spring 2020 Annual Fund Appeal (S27)'
donations.loc[(donations['close_date'] > '2019-11-19') & (donations['close_date'] < '2020-02-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Fall 2019 Annual Fund Appeal (S27)'
donations.loc[(donations['close_date'] > '2019-05-20') & (donations['close_date'] < '2019-08-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Spring 2019 Annual Fund Appeal (S26)'
donations.loc[(donations['close_date'] > '2018-11-25') & (donations['close_date'] < '2019-02-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Fall 2018 Annual Fund Appeal (S26)'
donations.loc[(donations['close_date'] > '2018-05-20') & (donations['close_date'] < '2018-08-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Spring 2018 Annual Fund Appeal (S25)'
donations.loc[(donations['close_date'] > '2017-11-10') & (donations['close_date'] < '2018-02-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Fall 2017 Annual Fund Appeal (S25)'
donations.loc[(donations['close_date'] > '2016-11-14') & (donations['close_date'] < '2017-02-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Fall 2016 Annual Fund Appeal (S24)'
donations.loc[(donations['close_date'] > '2015-11-14') & (donations['close_date'] < '2016-02-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Fall 2015 Annual Fund Appeal (S23)'
donations.loc[(donations['close_date'] > '2014-11-14') & (donations['close_date'] < '2015-02-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Fall 2014 Annual Fund Appeal (S22)'
donations.loc[(donations['close_date'] > '2020-04-30') & (donations['close_date'] < '2020-08-01') & (donations['type'] == 'Give OUT Day') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Give OUT Day (S27)'
donations.loc[((donations['fiscal_year'] == '2019') | (donations['close_date'] == '2019-07-19')) & (donations['type'] == 'Give OUT Day') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Give OUT Day (S26)'
donations.loc[((donations['fiscal_year'] == '2018') | (donations['close_date'] == '2020-03-10')) & (donations['type'] == 'Give OUT Day') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Give OUT Day (S25)'

In [ ]:
donations[donations['campaign'].isna()].reset_index(drop=True).head(20)

In [ ]:
donations.loc[(donations['close_date'] > '2020-06-30') & (donations['close_date'] < '2021-07-01') & (donations['campaign'].isna()), 'campaign'] = 'Outside of Campaign Period (S28)'
donations.loc[(donations['close_date'] > '2019-06-30') & (donations['close_date'] < '2020-07-01') & (donations['campaign'].isna()), 'campaign'] = 'Outside of Campaign Period (S27)'
donations.loc[(donations['close_date'] > '2018-06-30') & (donations['close_date'] < '2019-07-01') & (donations['campaign'].isna()), 'campaign'] = 'Outside of Campaign Period (S26)'
donations.loc[(donations['close_date'] > '2017-06-30') & (donations['close_date'] < '2018-07-01') & (donations['campaign'].isna()), 'campaign'] = 'Outside of Campaign Period (S25)'
donations.loc[(donations['close_date'] > '2016-06-30') & (donations['close_date'] < '2017-07-01') & (donations['campaign'].isna()), 'campaign'] = 'Outside of Campaign Period (S24)'
donations.loc[(donations['close_date'] > '2015-06-30') & (donations['close_date'] < '2016-07-01') & (donations['campaign'].isna()), 'campaign'] = 'Outside of Campaign Period (S23)'
donations.loc[(donations['close_date'] > '2014-06-30') & (donations['close_date'] < '2015-07-01') & (donations['campaign'].isna()), 'campaign'] = 'Outside of Campaign Period (S22)'

In [ ]:
donations.campaign.value_counts(dropna=False)

In [ ]:
donations.isna().sum()

As shown in output above, null values in all retained columns have been addressed.

**Explore donations df (with nulls handled) through visualizations**

In [ ]:
col_names = ['amount',
 'close_date',
 'fiscal_year',
 'last_donation_amount',
 'last_donation_date',
 'lifetime_donation_count',
 'lifetime_donation_total_amount',
 'lifetime_order_count',
 'lifetime_single_ticket_purchase_total',
 'lifetime_subscription_purchase_total',
 'amount_due',
 'amount_paid',
 'amount_donated_last_365_days',
 'amount_donated_last_fiscal_year',
 'amount_donated_this_fiscal_year',
 'total_amount',
 'order_count_last_#_days']

fig, ax = plt.subplots(len(col_names), figsize=(24,18))

for i, col_val in enumerate(col_names):

    sns.distplot(donations[col_val], hist=True, ax=ax[i])
    ax[i].set_title('Freq dist '+col_val, fontsize=10)
    ax[i].set_xlabel(col_val, fontsize=8)
    ax[i].set_ylabel('Count', fontsize=8)

plt.show()

In [ ]:
figure, ax = plt.subplots(6,2, figsize=(12,24))

#See the distrubution of the data

sns.distplot(donations['amount'],ax= ax[0,0])
sns.distplot(donations['last_donation_amount'],ax=ax[0,1])
sns.distplot(donations['lifetime_donation_count'],ax= ax[1,0])
sns.distplot(donations['lifetime_donation_total_amount'],ax= ax[1,1])
sns.distplot(donations['lifetime_single_ticket_purchase_total'],ax= ax[2,0])
sns.distplot(donations['lifetime_subscription_purchase_total'],ax=ax[2,1])
sns.distplot(donations['amount_donated_last_365_days'],ax= ax[3,0])
sns.distplot(donations['amount_donated_last_fiscal_year'],ax= ax[3,1])
sns.distplot(donations['amount_due'],ax= ax[4,0])
sns.distplot(donations['total_amount'],ax=ax[4,1])
sns.distplot(donations['lifetime_order_count'],ax= ax[5,0])
sns.distplot(donations['order_count_last_#_days'],ax= ax[5,1])

plt.show() #optional line -- eliminates strange text output before plots

In [ ]:
figure, ax = plt.subplots(8,2, figsize=(16,32))

#See the distrubution of the data

sns.countplot(donations['donation_record_type'],ax=ax[0,0])
sns.countplot(donations['account_record_type'],ax= ax[0,1])
sns.countplot(donations['fund'],ax= ax[1,0])
sns.countplot(donations['payment_type'],ax=ax[1,1])
sns.countplot(donations['type'],ax= ax[2,0])
sns.countplot(donations['stage'],ax= ax[2,1])
sns.countplot(donations['card_type'],ax=ax[3,0])
sns.countplot(donations['campaign'],ax= ax[3,1])
sns.countplot(donations['fiscal_quarter'],ax= ax[4,0])
sns.countplot(donations['fiscal_year'],ax=ax[4,1])
sns.countplot(donations['do_not_mail'],ax= ax[5,0])
sns.countplot(donations['email_opt_out'],ax= ax[5,1])
sns.countplot(donations['mailing_state'],ax=ax[6,0])
sns.countplot(donations['anonymous'],ax= ax[6,1])
sns.countplot(donations['mailing_city'],ax= ax[7,0])
 

In [ ]:
#visualize pairwise relationships among selected continuous features
cont_cols = ['amount',
 'close_date',
 'fiscal_year',
 'last_donation_amount',
 'last_donation_date',
 'lifetime_donation_count',
 'lifetime_donation_total_amount',
 'lifetime_order_count',
 'lifetime_single_ticket_purchase_total',
 'lifetime_subscription_purchase_total',
 'amount_due',
 'amount_paid',
 'amount_donated_last_365_days',
 'amount_donated_last_fiscal_year',
 'amount_donated_this_fiscal_year',
 'total_amount',
 'order_count_last_#_days']
sns.pairplot(donations[cont_cols])
plt.show()

In [ ]:
#Lets look at the funds on amount vs lifetime_order_count:

sns.lmplot(x="amount", y="lifetime_order_count", hue="fund", data=donations, palette = 'muted', height = 7)
plt.show(sns)

In [ ]:
sns.lmplot(x="amount", y="lifetime_order_count", hue="fiscal_year", data=donations, palette = 'muted', height = 7)
plt.show(sns)

In [ ]:
sns.lmplot(x="lifetime_order_count", y="amount", hue="email_opt_out", data=donations, palette = 'muted', height = 7)
plt.show(sns)

In [ ]:
#Lets look at correlation:

corr = donations[cont_cols].corr()

plt.figure(figsize=(12, 9))

sns.heatmap(corr, cmap = 'Wistia', annot= True)
plt.show(sns)

# Create time-series and account_id-based dfs for external analysis

In [ ]:
#show columns names to serve as reference in choosing which to retain
donations.columns

# Engineer Features for Time Series

In [ ]:
#Create a donation_recency feature indicating how many days since this donation was made
donations['donation_recency'] = (date.today() - donations['close_date'].dt.date).dt.days
donations['donation_recency'].head()

In [ ]:
#Create feature showing the number of days between this donation and the last
donations['days_since_last'] = (donations['last_donation_date'] - donations['close_date']).dt.days
donations['days_since_last'].head()

In [ ]:
#Create feature indicating days elapsed since between this/current donation and the corresponding donor's first
donations['days_since_first'] = (donations['close_date'] - donations['first_donation_date']).dt.days
donations['days_since_first'].head()

In [ ]:
#Create first donation indicator
donations['first_donation_flag'] = np.where(donations['close_date'] == donations['first_donation_date'], 1, 0)
donations['first_donation_flag'].head()

In [ ]:
#Create last donation indicator
donations['last_donation_flag'] = np.where(donations['close_date'] == donations['last_donation_date'], 1, 0)
donations['last_donation_flag'].head()

In [ ]:
#Create dictionary of aggregations needed to summarize data as time series.
aggregations = {
    'number_of_donations' : ('donation_id', 'size'),
    'don_total' : ('amount', 'sum'),
    'don_median' : ('amount', 'median'),
    'don_avg' : ('amount', 'mean'),
    'don_recency_median' : ('donation_recency', 'median'),
    'don_recency_avg' : ('donation_recency', 'mean'),
    'last_don_median' : ('last_donation_amount', 'median'),
    'last_don_avg' : ('last_donation_amount', 'median'),
    'avg_days_since_last' : ('days_since_last', 'mean'),
    'median_days_since_last' : ('days_since_last', 'median'),
    'avg_days_since_first' : ('days_since_first', 'mean'),
    'median_days_since_first' : ('days_since_first', 'median'),
    'number_firsts_on_this_date' : ('first_donation_flag', 'sum'),
    'number_lasts_on_this_date' : ('last_donation_flag', 'sum')
}

Create the donation time series dfs by grouping by close_date (as primary index) as well as other features for the higher level indicies

In [ ]:
donations.groupby(['close_date', 'fund']).agg(**aggregations).reset_index('fund').tail(30)

In [ ]:
don_ts = donations.groupby(['close_date', 'fiscal_year', 'fund', 'account_record_type', 'donation_record_type']).agg(**aggregations).reset_index(['fiscal_year', 'fund', 'account_record_type', 'donation_record_type'])
don_ts.tail(10)

In [ ]:
#Add campaign as a group and output df column
don_ts_wcamp = donations.groupby(['close_date', 'fiscal_year', 'fund', 'account_record_type', 'donation_record_type', 'campaign']).agg(**aggregations).reset_index(['fiscal_year', 'fund', 'account_record_type', 'donation_record_type', 'campaign'])
don_ts_wcamp.tail(10)

In [ ]:
#Add type as a group and output df column
don_ts_wtype = donations.groupby(['close_date', 'fiscal_year', 'fund', 'account_record_type', 'donation_record_type', 'type']).agg(**aggregations).reset_index(['fiscal_year', 'fund', 'account_record_type', 'donation_record_type', 'type'])
don_ts_wtype.tail(10)

In [ ]:
donations.groupby('close_date').agg(**aggregations).tail(10)


In [ ]:
don_ts_all = donations.groupby(['close_date', 'fiscal_year', 'campaign', 'fund', 'type', 'account_record_type', 'donation_record_type']).agg(**aggregations).reset_index(['fiscal_year', 'campaign', 'fund', 'type', 'account_record_type', 'donation_record_type'])
don_ts_all.tail(10)

In [ ]:
#Write the time series above to csv
don_ts_all.to_csv('daily_donations.csv')

**Come back to add development communication dates into times series as binary flags** 

In [ ]:
don_ts.columns

# Outlier Detection Analysis

In [ ]:
donations.loc[donations['amount'] < 15000].hist(column='amount', bins=30)

Because the frequency distribution plot of the amount variable is skewed (right) rather than approximately normal, outliers are likely better defined using criteria other than 3 std from mean. Perhaps 1.5(IRQ), percentile-based or z-score.

In [ ]:
def percentile_based_outlier(data, threshold=95):
    diff = (100 - threshold) / 2
    minval, maxval = np.percentile(data, [diff, 100 - diff])
    return (data < minval) | (data > maxval)

col_names = ['amount',
 'last_donation_amount',
 'lifetime_donation_count',
 'lifetime_donation_total_amount',
 'lifetime_order_count',
 'lifetime_single_ticket_purchase_total',
 'lifetime_subscription_purchase_total',
 'amount_due',
 'amount_paid',
 'amount_donated_last_365_days',
 'amount_donated_last_fiscal_year',
 'amount_donated_this_fiscal_year',
 'group_sales_total',
 'total_amount',
 'order_count_last_#_days']

fig, ax = plt.subplots(len(col_names), figsize=(8,40))

for i, col_val in enumerate(col_names):
    x = donations[col_val]
    sns.distplot(x, ax=ax[i], rug=True, hist=False)
    outliers = x[percentile_based_outlier(x)]
    ax[i].plot(outliers, np.zeros_like(outliers), 'ro', clip_on=False)

    ax[i].set_title('Outlier detection - {}'.format(col_val), fontsize=10)
    ax[i].set_xlabel(col_val, fontsize=8)

plt.show()

In [ ]:
#Define outlier donation amounts as those below quartile 1 - 1.5*IQR or above quartile 3 + 1.5*IQR
IQR = donations.describe().at['75%', 'amount'] - donations.describe().at['25%', 'amount']
lthreshold = donations.describe().at['25%', 'amount'] - 1.5*IQR
uthreshold = donations.describe().at['75%', 'amount'] + 1.5*IQR

#Print range of 'normal' data (i.e. range including non-outlier amounts):
print (lthreshold, uthreshold)

In [ ]:
#With no donations below zero, we identify and address outliers as those amounts > 287.645
#Impute outlier amounts from the dataframe with the median of the top half of the distribution
donations['amount'] = np.where(donations['amount'] > uthreshold, donations.describe().at['75%', 'amount'], donations['amount'])

In [ ]:
#Plot new frequency distribution of amount column
donations.hist(column='amount', bins=30)

In [ ]:
plt.hist(donations['amount'], density=True, alpha=.5)
plt.title("Frequency Distribution of Amount Variable (with Outlier Imputation)")
plt.xlabel("Donation Amount Bins")
plt.ylabel("Probability")

In [ ]:
#Plot cumulative distribution of amount column
plt.hist(donations['amount'], density=True, alpha=.5, cumulative=True)

In [ ]:
.87602906-.69854722

The largest step in the cumulative probability distribution of 17.75%, which exists between the 100 bar and the next one, suggests a substantial concentration of unique donations near the stewardship program entry level amount of $125. 

In [ ]:
# #Define outlier donation amount as any amount greater than 3 standard deviations from the mean
# outlier_threshold = donations.describe().at['mean', 'amount'] + 3*(donations.describe().at['std', 'amount'])

# #Impute outlier amounts from the dataframe with the col median.
# donations['amount'] = np.where(donations['amount'] > outlier_threshold, donations.amount.median(), donations['amount'])

In [ ]:
donations.describe().T

**Explore the bivariate relationships of the continuous variables**

Separate continuous and categorical features for exploration.

In [ ]:
cont_cols = list(donations.describe().T.index)
cont_cols

In [ ]:
#Create continuous dataframe
cont = donations.loc[:, cont_cols].dropna().reset_index(drop=True)
cont.drop(['do_not_mail', 'email_opt_out', 'anonymous', 'group_sales_total', 'amount_matched'], axis=1, inplace=True)
cont

In [ ]:
#Select remaining variables for cats df (categorical variables)
#features = donations.columns
#cont_vars = cont.columns
#cat_vars = [f for f in features if f not in cont_vars]

In [ ]:
#Create cont df
#cats = donations.loc[:, cat_vars]

#Add in the continuous target variable: 'amount'
#cats['amount'] = cont['amount']

#Review head records
#cats.head()

In [ ]:
sns.pairplot(cont)

Amount Column

In [ ]:
#Visualize donations over time
dons = donations.plot.scatter('close_date', 'amount')

In [ ]:
sns.boxplot(x=donations['amount'])

Boxplot illustrates our efficacy in imputing amount outliers

In [ ]:
donations_by_date = donations.groupby('close_date')['donation_id'].count()
donations_by_date.head()

In [ ]:
donations_by_date.tail(25)

In [ ]:
#Plot the daily donation volume

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.lineplot(x=donations_by_date.index, y=donations_by_date.values, color='purple')

#Assign axis labels:
plt.ylabel('Number of Donations', fontsize=14)
plt.xlabel('Date', fontsize=14)
plt.title('Volume of All Donations by Close Date')

In [ ]:
#Which date over the analysis period had the largest donation volume?
donations.close_date.mode()

#04-19-2018 makes sense as it was our first giveoutday

In [ ]:
# #Fill nan in donations['amount'] with 0
# donations.loc[donations['amount'].isna(), 'amount'] = 0
# #Check it
# donations[donations['donation_record_type'] == 'Pledge']

In [ ]:
# #Convert nan in fund column (except for pledges) to Annual Fund.
# #Also, replace other AF categories with 'Annual Fund'
# donations.loc[(donations['fund'].isna()) & (donations['donation_record_type'] != 'Pledge') & (donations['account_record_type'].isin(["Individual", 'Household'])), 'fund'] = "Annual Fund"

In [ ]:
#donations.loc[(donations(donations['fund'].isin(["Annual Fund Historical", 'ANN']) & ['donation_record_type'] != 'Pledge') & (donations['account_record_type'] != 'Business') & (donations['fund'].isna()), 'fund'] = 'Annual Fund'

In [ ]:
#Group levels referencing the same fund under common levels
# donations_no_outlier_amts.loc[(donations_no_outlier_amts['fund'].isin(["25th Anniversary Campaign Fund", '25th Anniversary Fund', '25TH'])), 'fund'] = "25th Anniversary Capital Campaign"
# donations_no_outlier_amts.loc[(donations_no_outlier_amts['fund'] == 'NEXT'), 'fund'] = "Next Stage Campaign"
# donations_no_outlier_amts.loc[(donations_no_outlier_amts['fund'] == 'John Knapp -Tim Gillham Fund'), 'fund'] = "New Works (Knapp-Gillham) Fund"
# donations_no_outlier_amts.loc[(donations_no_outlier_amts['fund'].isin(["Education Program Fund", 'EDUC'])), 'fund'] = "Education Fund"
# donations_no_outlier_amts.loc[(donations_no_outlier_amts['fund'] == 'MMP'), 'fund'] = "Mary Morton Parsons Challenge Grant Match"

In [ ]:
#Show all donation totals by fiscal year
donations_by_fy = donations.groupby('fiscal_year')['amount'].sum()
donations_by_fy

In [ ]:
#Plot the FY donation totals

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.lineplot(x=donations_by_fy.index, y=donations_by_fy.values, color='magenta')

#Assign axis labels:
plt.ylabel('Donation Total ($)', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)

In [ ]:
#Plot the FY donation volume

#set style
#sns.set(style='darkgrid')

#Adjust the plot size
#plt.figure(figsize=[15,6])

#Line Plot
#sns.lineplot(x=donations.index, y=donations.values, color='magenta')

#Assign axis labels:
#plt.ylabel('Donation Total ($)', fontsize=14)
#plt.xlabel('Fiscal Year', fontsize=14)

In [ ]:
#Show donation totals by fund and fiscal year
donations_by_fy_fund = donations.groupby(['fund','fiscal_year'])['amount'].sum()
donations_by_fy_fund

In [ ]:
#Visualize annual fund donations by date
af_donations_by_close_date = donations.loc[(donations['fund'] == 'Annual Fund')].groupby('close_date')['amount'].sum()
af_donations_by_close_date.head()

In [ ]:
#Plot the daily annual fund donation totals

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.lineplot(x=af_donations_by_close_date.index, y=af_donations_by_close_date.values, color='magenta')

#Assign axis labels:
plt.ylabel('Donation Total ($)', fontsize=14)
plt.xlabel('Close Date', fontsize=14)
plt.title('Annual Fund Donation Totals Per Day')

In [ ]:
#Visualize mean annual fund donations by FY
af_donations_mean_by_fy = donations[(donations['fund'] == 'Annual Fund')].groupby('fiscal_year')['amount'].mean()
af_donations_mean_by_fy

In [ ]:
#Plot the annual fund means by fiscal year

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.barplot(x=af_donations_mean_by_fy.index, y=af_donations_mean_by_fy.values, color='green')

#Assign axis labels:
plt.ylabel('Mean Donations ($)', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)
plt.title('Annual Fund Donation Means By Fiscal Year')

In [ ]:
pd.DataFrame(af_donations_mean_by_fy).T

In [ ]:
#Visualize annual fund donation volume by FY
af_donations_vol_by_fy = donations[(donations['fund'] == 'Annual Fund')].groupby('fiscal_year')['amount'].count()
af_donations_vol_by_fy

In [ ]:
#Plot the annual fund volume by fiscal year

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.barplot(x=af_donations_vol_by_fy.index, y=af_donations_vol_by_fy.values, color='purple')

#Assign axis labels:
plt.ylabel('Number of Donors', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)
plt.title('Annual Fund Donation Volume By Fiscal Year')

In [ ]:
#AF donation totals by FY
af_totals_fy = donations[(donations['fund'] == 'Annual Fund')].groupby('fiscal_year')['amount'].sum()
af_totals_fy

In [ ]:
#Plot the annual fund totals by fiscal year

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.barplot(x=af_totals_fy.index, y=af_totals_fy.values, color='purple')

#Assign axis labels:
plt.ylabel('Number of Donors', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)
plt.title('Annual Fund Donation Totals By Fiscal Year')

# Stewardship Program Analysis

Create stewardship level columns

In [ ]:
#Only Annual Fund dontations made by Household or Individual donors are included in stewardship program totals
donations['stp_total_s28'] = donations.loc[(donations['close_date'] >= '2019-07-01') & (donations['fund'] == 'Annual Fund') & (donations['account_record_type'].isin(['Household', 'Individual']))].groupby('account_id')['amount'].transform('sum')
donations['stp_total_s27'] = donations.loc[(donations['close_date'] >= '2018-07-01') & (donations['close_date'] <= '2020-06-30') & (donations['fund'] == 'Annual Fund') & (donations['account_record_type'].isin(['Household', 'Individual']))].groupby('account_id')['amount'].transform('sum')
donations['stp_total_s26'] = donations.loc[(donations['close_date'] >= '2017-07-01') & (donations['close_date'] <= '2019-06-30') & (donations['fund'] == 'Annual Fund') & (donations['account_record_type'].isin(['Household', 'Individual']))].groupby('account_id')['amount'].transform('sum')
donations['stp_total_s25'] = donations.loc[(donations['close_date'] >= '2016-07-01') & (donations['close_date'] <= '2018-06-30') & (donations['fund'] == 'Annual Fund') & (donations['account_record_type'].isin(['Household', 'Individual']))].groupby('account_id')['amount'].transform('sum')
donations['stp_total_s24'] = donations.loc[(donations['close_date'] >= '2015-07-01') & (donations['close_date'] <= '2017-06-30') & (donations['fund'] == 'Annual Fund') & (donations['account_record_type'].isin(['Household', 'Individual']))].groupby('account_id')['amount'].transform('sum')
donations['stp_total_s23'] = donations.loc[(donations['close_date'] >= '2014-07-01') & (donations['close_date'] <= '2016-06-30') & (donations['fund'] == 'Annual Fund') & (donations['account_record_type'].isin(['Household', 'Individual']))].groupby('account_id')['amount'].transform('sum')

In [ ]:
#Fill NaNs with max value for that account_id shown in other rows for that particular stp_total_sXX column 
stp_total_cols = ['stp_total_s28', 'stp_total_s27', 'stp_total_s26', 'stp_total_s25', 'stp_total_s24', 'stp_total_s23']
for col in stp_total_cols:
    donations.loc[donations[col].isnull(), col] = donations.groupby('account_id')[col].transform('max')

In [ ]:
stp_total_cols.append('account_id')

In [ ]:
#Review cols just created
donations.loc[:, stp_total_cols]

In [ ]:
#Now, fill remaining nan in stp_total_sXX cols with 0 to simplify ufunc (stp_level_generator) below:  
donations[['stp_total_s28', 'stp_total_s27', 'stp_total_s26', 'stp_total_s25', 'stp_total_s24', 'stp_total_s23']] = donations.loc[:,['stp_total_s28', 'stp_total_s27', 'stp_total_s26', 'stp_total_s25', 'stp_total_s24', 'stp_total_s23']].fillna(0)

In [ ]:
#Confirm accuracy by checking output for one account_id
donations.loc[donations['account_id'] == '0013600000VKRp8', ['stp_total_s28', 'stp_total_s27', 'stp_total_s26', 'stp_total_s25', 'stp_total_s24', 'stp_total_s23']]

In [ ]:
#Double check that nulls are eliminated for new cols
donations.isna().sum()

# Translate stewardship amount by season into stewardship level by account_id

In [ ]:
def stp_level_generator(series):
    if series == 0:
        return 'Did Not Give'
    elif (series > 0) & (series < 125):
        return 'Pre-Steward'
    elif (series >= 125) & (series < 250):
        return 'Supporter'
    elif (series >= 250) & (series < 500):
        return 'Investor'
    elif (series >= 500) & (series < 1000):
        return 'Underwriter'
    elif (series >= 1000) & (series < 2500):
        return 'Performer'
    elif (series >= 2500) & (series < 5000):
        return 'Director'
    elif (series >= 5000):
        return 'Producer'

In [ ]:
#Create new dataframe with account_id, stp_total_sX cols and corresponding stp_level cols: 
stp_amt_list = ['stp_total_s28', 'stp_total_s27', 'stp_total_s26', 'stp_total_s25', 'stp_total_s24', 'stp_total_s23']
levels_by_season_df = pd.DataFrame()
levels_by_season_df['account_id'] = donations['account_id']

for col in stp_amt_list:
    levels_by_season_df[col] = donations[col]
    col_name = col[:3] + col[9:] + '_level'
    levels_by_season_df[col_name] = (donations[col].apply(stp_level_generator))

In [ ]:
levels_by_season_df.head()

In [ ]:
#Remove duplicate rows based on account_id to show each donor's stewardship levels across the most recent 6 seasons
levels_by_season_df = levels_by_season_df.drop_duplicates(subset='account_id', keep='first').reset_index(drop='True')

In [ ]:
#Check that all stewardship levels are present in each season
level_cols = ['stp_s23_level', 'stp_s24_level', 'stp_s25_level', 'stp_s26_level', 'stp_s27_level', 'stp_total_s28']
for col in level_cols:
    print(levels_by_season_df[col].value_counts())

The producer level is missing from season 25. Add it below when creating that series.

In [ ]:
stp_donors_s27 = (levels_by_season_df.groupby('stp_s27_level')['account_id'].count()).rename('stp_s27_level')
stp_donors_s27

In [ ]:
stp_donors_s26 = levels_by_season_df.groupby('stp_s26_level')['account_id'].count().rename('stp_s26_level')

In [ ]:
stp_donors_s25 = levels_by_season_df.groupby('stp_s25_level')['account_id'].count().rename('stp_s25_level')

In [ ]:
#stp_s25_level is has no 'Producer' level records. Add this level to the index with a value of 0.
stp_donors_s25.loc['Producer'] = 0

In [ ]:
stp_donors_s25

In [ ]:
stp_donors_s24 = levels_by_season_df.groupby('stp_s24_level')['account_id'].count().rename('stp_s24_level')

In [ ]:
#stp_s24_level is has no 'Producer' level records. Add this level to the index with a value of 0.
stp_donors_s24.loc['Producer'] = 0

In [ ]:
stp_donors_s23 = levels_by_season_df.groupby('stp_s23_level')['account_id'].count().rename('stp_s23_level')

In [ ]:
#stp_s23_level is has no 'Producer' level records. Add this level to the index with a value of 0.
stp_donors_s23.loc['Producer'] = 0

# Visualize change in level size across time (seasons)

In [ ]:
#Concatenate the series above to facilitate visualizing stewardship levels by season
stp_level_dist = (pd.concat([stp_donors_s23, stp_donors_s24, stp_donors_s25, stp_donors_s26, stp_donors_s27], axis=1, join='inner', copy=False))

In [ ]:
stp_level_dist

In [ ]:
#Create bar chart showing donor volume by stewardship level for season 23
stp_23 = (levels_by_season_df.groupby('stp_s23_level')['account_id'].count().to_frame().iloc[1:]).plot(kind='bar', stacked=True)

In [ ]:
ax = stp_level_dist.plot(kind='bar', stacked=True)

Change row index for chart above so that the levels are in proper order.

In [ ]:
stp_level_dist = stp_level_dist.loc[['Did Not Give', 'Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer']]
stp_level_dist.index

In [ ]:
#Replot bar chart above with proper level order
ax = stp_level_dist.plot(kind='bar', stacked=True)

In [ ]:
stp_level_dist.T

In [ ]:
#Change the row labels to fix improve the bar charts
stp_level_dist.T.set_index(pd.Index(['Season 23', 'Season 24', 'Season 25', 'Season 26', 'Season 27']))

In [ ]:
ax = stp_level_dist.T.set_index(pd.Index(['Season 23', 'Season 24', 'Season 25', 'Season 26', 'Season 27'])).plot(kind='bar', stacked=True, title='Stewardship Level Volume by Season')

In [ ]:
#Replot above without Did Not Give group in each season
stp_levels = stp_level_dist[1:].T.set_index(pd.Index(['Season 23', 'Season 24', 'Season 25', 'Season 26', 'Season 27'])).plot(kind='bar', stacked=True, title='Stewardship Level Volume by Season')

In [ ]:
#Transform df to show stewardship level donor counts by season, including only the actual stewardship levels
stp_level_dist.T.set_index(pd.Index(['Season 23', 'Season 24', 'Season 25', 'Season 26', 'Season 27'])).loc[:,['Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer']]

In [ ]:
stp_stacked_bars = stp_level_dist.T.set_index(pd.Index(['Season 23', 'Season 24', 'Season 25', 'Season 26', 'Season 27'])).loc[:,['Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer']].plot(kind='bar', stacked=True, title='Stewardship Levels Largely Grow Over Time')

In [ ]:
stp_level_counts = stp_level_dist.T.set_index(pd.Index(['Season 23', 'Season 24', 'Season 25', 'Season 26', 'Season 27'])).loc[:,['Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer']]
stp_level_counts

In [ ]:
# set width of bar
barWidth = 0.25
 
# set height of bar
bars1 = list(stp_level_counts.T.values[0])
bars2 = list(stp_level_counts.T.values[1])
bars3 = list(stp_level_counts.T.values[2])
bars4 = list(stp_level_counts.T.values[3])
bars5 = list(stp_level_counts.T.values[4])
bars6 = list(stp_level_counts.T.values[5])
 
# Set position of bar on X axis
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]
r5 = [x + barWidth for x in r4]
r6 = [x + barWidth for x in r5]

print(r1, r2, r3, r4, r5, r6)

In [ ]:
#Visualize stewardship level counts by season as a grouped barplot.

# set width of bar
barWidth = 0.150 #Resolved overlapping bars by reducing the width from .25 to .15
 
# set height of bar
bars1 = list(stp_level_counts.T.values[0])
bars2 = list(stp_level_counts.T.values[1])
bars3 = list(stp_level_counts.T.values[2])
bars4 = list(stp_level_counts.T.values[3])
bars5 = list(stp_level_counts.T.values[4])
bars6 = list(stp_level_counts.T.values[5])
 
# Set position of bar on X axis
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]
r5 = [x + barWidth for x in r4]
r6 = [x + barWidth for x in r5]
 
# Make the plot
plt.bar(r1, bars1, color='yellow', width=barWidth, edgecolor='white', label='Supporter')
plt.bar(r2, bars2, color='lightblue', width=barWidth, edgecolor='white', label='Investor')
plt.bar(r3, bars3, color='orange', width=barWidth, edgecolor='white', label='Underwriter')
plt.bar(r4, bars4, color='navy', width=barWidth, edgecolor='white', label='Performer')
plt.bar(r5, bars5, color='red', width=barWidth, edgecolor='white', label='Director')
plt.bar(r6, bars6, color='purple', width=barWidth, edgecolor='white', label='Producer')

# Add xticks on the middle of the group bars
plt.xlabel('Season', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(bars1))], list(stp_level_counts.index))
 
# Add title, Create legend and Show graphic
plt.title('Stewardship Level Volumes Largely Grow Over Time')
plt.legend()
plt.show()


In [ ]:
stp_donor_counts = stp_level_dist.T.set_index(pd.Index(['Season 23', 'Season 24', 'Season 25', 'Season 26', 'Season 27'])).loc[:,['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer']]
stp_donor_counts

In [ ]:
#Visualize stewardship donor counts by season as a grouped barplot.

# set width of bar
barWidth = 0.10 
 
# set height of bar
bars1 = list(stp_donor_counts.T.values[0])
bars2 = list(stp_donor_counts.T.values[1])
bars3 = list(stp_donor_counts.T.values[2])
bars4 = list(stp_donor_counts.T.values[3])
bars5 = list(stp_donor_counts.T.values[4])
bars6 = list(stp_donor_counts.T.values[5])
bars7 = list(stp_donor_counts.T.values[6])
 
# Set position of bar on X axis
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]
r5 = [x + barWidth for x in r4]
r6 = [x + barWidth for x in r5]
r7 = [x + barWidth for x in r6]
 
# Make the plot
plt.bar(r1, bars1, color='magenta', width=barWidth, edgecolor='white', label='Pre-Steward')
plt.bar(r2, bars2, color='yellow', width=barWidth, edgecolor='white', label='Supporter')
plt.bar(r3, bars3, color='lightblue', width=barWidth, edgecolor='white', label='Investor')
plt.bar(r4, bars4, color='orange', width=barWidth, edgecolor='white', label='Underwriter')
plt.bar(r5, bars5, color='navy', width=barWidth, edgecolor='white', label='Performer')
plt.bar(r6, bars6, color='red', width=barWidth, edgecolor='white', label='Director')
plt.bar(r7, bars7, color='purple', width=barWidth, edgecolor='white', label='Producer')

# Add xticks on the middle of the group bars
plt.xlabel('Season', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(bars1))], list(stp_level_counts.index))
 
# Add title, Create legend and Show graphic
plt.title('Stewardship Donor Volumes Largely Grow Over Time')
plt.legend()
plt.show()



Run a statistical test to determine if the number of donors in each level is statistically significantly higher than each of them in the previous season.

Examine growth rates in levels over time

In [ ]:
stp_counts = stp_level_dist.T.set_index(pd.Index(['Season 23', 'Season 24', 'Season 25', 'Season 26', 'Season 27']))
stp_counts

In [ ]:
#Define function to generate growth rates in levels over time:

def stp_growth_rates_by_level(df, r, row_name_list):
    """
Params
    ------
    df : pandas.DataFrame
        dataframe with columns as stewardship levels and row index as seasons
    r : integer
        integer referencing max row index of the df

    Returns
    -------
    np.arrary
        Returns a numpy array with one fewer row dimension than the input df, 
        Each element shows the rounded percent change in each stewardship level value over the previous season
        
    """
    a = np.round_((100*(df.to_numpy()[1:,:] - df.to_numpy()[:r,:])/ df.to_numpy()[:r,:]), 2)
    a[np.isinf(a)] = 0
    x = pd.DataFrame(data=a,
                index=row_name_list,
                columns=df.columns)
    return x

In [ ]:
perc_change_level_vol = stp_growth_rates_by_level(stp_counts, 4, ['% Change in Season 24', '% Change in Season 25', '% Change in Season 26', '% Change in Season 27'])
perc_change_level_vol

In [ ]:
#Visualize percent change in stewardship level volume by season as a grouped barplot.

# set width of bar
barWidth = 0.10 
 
# set height of bar
bars1 = list(perc_change_level_vol.T.values[0])
bars2 = list(perc_change_level_vol.T.values[1])
bars3 = list(perc_change_level_vol.T.values[2])
bars4 = list(perc_change_level_vol.T.values[3])
bars5 = list(perc_change_level_vol.T.values[4])
bars6 = list(perc_change_level_vol.T.values[5])
bars7 = list(perc_change_level_vol.T.values[6])
bars8 = list(perc_change_level_vol.T.values[6])
 
# Set position of bar on X axis
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]
r5 = [x + barWidth for x in r4]
r6 = [x + barWidth for x in r5]
r7 = [x + barWidth for x in r6]
r8 = [x + barWidth for x in r7]
 
# Make the plot
plt.bar(r1, bars1, color='teal', width=barWidth, edgecolor='white', label='Did Not Give')
plt.bar(r2, bars2, color='magenta', width=barWidth, edgecolor='white', label='Pre-Steward')
plt.bar(r3, bars3, color='yellow', width=barWidth, edgecolor='white', label='Supporter')
plt.bar(r4, bars4, color='lightblue', width=barWidth, edgecolor='white', label='Investor')
plt.bar(r5, bars5, color='orange', width=barWidth, edgecolor='white', label='Underwriter')
plt.bar(r6, bars6, color='navy', width=barWidth, edgecolor='white', label='Performer')
plt.bar(r7, bars7, color='red', width=barWidth, edgecolor='white', label='Director')
plt.bar(r8, bars8, color='purple', width=barWidth, edgecolor='white', label='Producer')

# Add xticks on the middle of the group bars
plt.xlabel('Season', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(bars1))], list(perc_change_level_vol.index), rotation=45)
plt.ylabel('Percent Change in Level Size', fontweight='bold')
 
# Add title, Create legend and Show graphic
plt.title('Stewardship Levels Sizes Tend to Grow Each Season')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()


In [ ]:
#function to create a statewise stacked plot of positive and negative cases
#def pandas_plot(covid_df):
#    ax = covid_df.iloc[:,[0,7]].plot(kind='bar',stacked=True,title = 'US Covid19 Stats By State as of '+max(covid_df['dateModified']),rot=0)
#    for lbl in ax.patches:
#        ax.annotate("{:.0f}".format(int(lbl.get_height())),\
#                    (lbl.get_x()+.25, lbl.get_height()+lbl.get_y()),\
#                    verticalalignment='bottom', horizontalalignment='center', \
#                    fontsize=7, color='black')
        
#if __name__ == '__main__':
#    state_list = ['NY','WA','CA','NJ'] #define list of states for which details are needed
#    covid_df = api_dataload(state_list) #pass the list of states to the api_dataloaf function
#    pandas_plot(covid_df) #use the consolidated covid_df to create stacked plot using pandas_plot function

# Visualize change in donation totals across time

Create a dataframe of donation totals & means by stewardship level over time.

In [ ]:
stp_totals_s23 = levels_by_season_df.groupby('stp_s23_level')['stp_total_s23'].sum().rename('stp_totals_s23').sort_values()
stp_totals_s23

In [ ]:
stp_means_s23 = levels_by_season_df.groupby('stp_s23_level')['stp_total_s23'].mean().rename('stp_means_s23').sort_values()
stp_means_s23

In [ ]:
stp_totals_s24 = levels_by_season_df.groupby('stp_s24_level')['stp_total_s24'].sum().rename('stp_totals_s24').sort_values()
stp_totals_s24

In [ ]:
stp_means_s24 = levels_by_season_df.groupby('stp_s24_level')['stp_total_s24'].mean().rename('stp_means_s24').sort_values()
stp_means_s24

In [ ]:
stp_totals_s25 = levels_by_season_df.groupby('stp_s25_level')['stp_total_s25'].sum().rename('stp_totals_s25').sort_values()
stp_totals_s25

In [ ]:
stp_means_s25 = levels_by_season_df.groupby('stp_s25_level')['stp_total_s25'].mean().rename('stp_means_s25').sort_values()
stp_means_s25

In [ ]:
stp_totals_s26 = levels_by_season_df.groupby('stp_s26_level')['stp_total_s26'].sum().rename('stp_totals_s26').sort_values()
stp_totals_s26

In [ ]:
stp_means_s26 = levels_by_season_df.groupby('stp_s26_level')['stp_total_s26'].mean().rename('stp_means_s26').sort_values()
stp_means_s26

In [ ]:
stp_totals_s27 = levels_by_season_df.groupby('stp_s27_level')['stp_total_s27'].sum().rename('stp_totals_s27').sort_values()
stp_totals_s27

In [ ]:
stp_means_s27 = levels_by_season_df.groupby('stp_s27_level')['stp_total_s27'].mean().rename('stp_means_s27').sort_values()
stp_means_s27

# Determine which accounts made any annual fund contributions in each season:

In [ ]:
#Season 27 AF Donors
(levels_by_season_df[levels_by_season_df['stp_s27_level'] != 'Did Not Give'].reset_index(drop='True')).loc[:,['account_id','stp_s27_level']]

In [ ]:
#Season 26 AF Donors
(levels_by_season_df[levels_by_season_df['stp_s26_level'] != 'Did Not Give'].reset_index(drop='True')).loc[:,['account_id','stp_s26_level']]

In [ ]:
#Season 25 AF Donors
(levels_by_season_df[levels_by_season_df['stp_s25_level'] != 'Did Not Give'].reset_index(drop='True')).loc[:,['account_id','stp_s25_level']]

In [ ]:
#Season 24 AF Donors
(levels_by_season_df[levels_by_season_df['stp_s24_level'] != 'Did Not Give'].reset_index(drop='True')).loc[:,['account_id','stp_s24_level']]

In [ ]:
#Season 23 AF Donors
(levels_by_season_df[levels_by_season_df['stp_s23_level'] != 'Did Not Give'].reset_index(drop='True')).loc[:,['account_id','stp_s23_level']]

In [ ]:
#Create annual_fund dictionary where each key stores the account_ids of donors who gave during that season
level_cols = ['stp_s27_level', 'stp_s26_level', 'stp_s25_level', 'stp_s24_level', 'stp_s23_level']
annual_fund = {}
for col in level_cols:
    annual_fund[col[4:7]+'_donors'] = (levels_by_season_df[levels_by_season_df[col] != 'Did Not Give'].reset_index(drop='True')).loc[:,['account_id',col]]

In [ ]:
#View the annual fund dictionary, which shows donors levels by season (excluding those who did not give during a given season)
annual_fund

In [ ]:
annual_fund.values()

In [ ]:
#How many accounts donated to the annual fund in each season (s27-s23):
af_donor_keys = ['s27_donors', 's26_donors', 's25_donors', 's24_donors', 's23_donors']
for key in af_donor_keys:
    print(len(annual_fund[key]['account_id'].values))

# Stewardship Donor Retention: accounts retained as annual fund donors from season to season (s24-s27)

In [ ]:
#Which season 26 donors were retained in season 27?
retained_s27 = [a for a in annual_fund['s27_donors']['account_id'].values if a in annual_fund['s26_donors']['account_id'].values]

In [ ]:
#Season 27 donor retention rate:
round(len(retained_s27)/len(annual_fund['s27_donors']['account_id'].values)*100, 2)

In [ ]:
#Which season 25 donors were retained in season 26?
retained_s26 = [a for a in annual_fund['s26_donors']['account_id'].values if a in annual_fund['s25_donors']['account_id'].values]

In [ ]:
#Season 26 donor retention rate:
round(len(retained_s26)/len(annual_fund['s26_donors']['account_id'].values)*100, 2)

In [ ]:
#Which season 24 donors were retained in season 25?
retained_s25 = [a for a in annual_fund['s25_donors']['account_id'].values if a in annual_fund['s24_donors']['account_id'].values]

In [ ]:
#Season 25 donor retention rate:
round(len(retained_s25)/len(annual_fund['s25_donors']['account_id'].values)*100, 2)

In [ ]:
#Which season 23 donors were retained in season 24?
retained_s24 = [a for a in annual_fund['s24_donors']['account_id'].values if a in annual_fund['s23_donors']['account_id'].values]

In [ ]:
#Season 24 donor retention rate:
round(len(retained_s24)/len(annual_fund['s24_donors']['account_id'].values)*100, 2)

In [ ]:
#How many donors did we retain season over season (count number of donors in the 4 lists created above):
len(retained_s27), len(retained_s26), len(retained_s25), len(retained_s24)

In [ ]:
#Retained donor growth rates:
retained_donor_growth_s25 = round((len(retained_s25) - len(retained_s24))/len(retained_s24)*100, 2)
f'Retained Annual Fund Donor Growth Rate s25 versus s24 is {retained_donor_growth_s25}%'

In [ ]:
retained_donor_growth_s26 = round((len(retained_s26) - len(retained_s25))/len(retained_s25)*100, 2)
f'Retained Annual Fund Donor Growth Rate s26 versus s25 is {retained_donor_growth_s26}%'

In [ ]:
retained_donor_growth_s27 = round((len(retained_s27) - len(retained_s26))/len(retained_s26)*100, 2)
f'Retained Annual Fund Donor Growth Rate s27 versus s26 is {retained_donor_growth_s27}%'

In [ ]:
#How many stewardship donors gave to the annual fund in s23, s24 and s25
retained_consecutively_2 = list(set.intersection(*map(set, [annual_fund['s25_donors']['account_id'].values, annual_fund['s24_donors']['account_id'].values, annual_fund['s23_donors']['account_id'].values])))
len(retained_consecutively_2)

In [ ]:
#How many stewardship donors gave to the annual fund in s23, s24, s25 and s26
retained_consecutively_3 = list(set.intersection(*map(set, [annual_fund['s26_donors']['account_id'].values, annual_fund['s25_donors']['account_id'].values, annual_fund['s24_donors']['account_id'].values, annual_fund['s23_donors']['account_id'].values])))
len(retained_consecutively_3)

In [ ]:
#How many stewardship donors gave to the annual fund in s23, s24, s25, s26 and s27
retained_consecutively_4 = list(set.intersection(*map(set, [annual_fund['s27_donors']['account_id'].values, annual_fund['s26_donors']['account_id'].values, annual_fund['s25_donors']['account_id'].values, annual_fund['s24_donors']['account_id'].values, annual_fund['s23_donors']['account_id'].values])))
len(retained_consecutively_4)

# Acquisition / Reactivation Metrics

In [ ]:
#Which accounts were acquired/reactivated in season 27?
new_s27 = [a for a in annual_fund['s27_donors']['account_id'].values if a not in annual_fund['s26_donors']['account_id'].values]
#How many?
len(new_s27)

In [ ]:
#Percent of season 27 donors that were newly acquired or reactivated donors:
round(((len(new_s27)/len(annual_fund['s27_donors']['account_id'].values))*100),2)

In [ ]:
#Which accounts were acquired/reactivated in season 26?
new_s26 = [a for a in annual_fund['s26_donors']['account_id'].values if a not in annual_fund['s25_donors']['account_id'].values]
#How many?
len(new_s26)

In [ ]:
#Percent of season 26 donors that were newly acquired or reactivated donors:
round(((len(new_s26)/len(annual_fund['s26_donors']['account_id'].values))*100),2)

In [ ]:
#Which accounts were acquired/reactivated in season 25?
new_s25 = [a for a in annual_fund['s25_donors']['account_id'].values if a not in annual_fund['s24_donors']['account_id'].values]
#How many?
len(new_s25)

In [ ]:
#Percent of season 25 donors that were newly acquired or reactivated donors:
round(((len(new_s25)/len(annual_fund['s25_donors']['account_id'].values))*100),2)

In [ ]:
#Which accounts were acquired/reactivated in season 24?
new_s24 = [a for a in annual_fund['s24_donors']['account_id'].values if a not in annual_fund['s23_donors']['account_id'].values]
#How many?
len(new_s24)

In [ ]:
#Percent of season 24 donors that were newly acquired or reactivated donors:
round(((len(new_s24)/len(annual_fund['s24_donors']['account_id'].values))*100),2)

# Churn Metrics: Annual Fund Donor Accounts who Churned Season over Season (s24-s27) 

Here, we're examining donors who churned defined as the specific donors who gave last season but not this season.
Churn rate = number who gave last season but not this season / total number who gave last season

In [ ]:
#Which season 26 donors churned in season 27?
churn_s27 = [a for a in annual_fund['s26_donors']['account_id'].values if a not in annual_fund['s27_donors']['account_id'].values]
#How many churned?
len(churn_s27)

In [ ]:
#Season 27 churn rate (based on specific donors):
churn_rate_s27 = round(len(churn_s27)/len(annual_fund['s26_donors']['account_id'].values)*100, 2)
churn_rate_s27

In [ ]:
#Season 27 churn rate (based on donor counts):
round(((len(annual_fund['s26_donors']['account_id'].values) - len(annual_fund['s27_donors']['account_id'].values))/len(annual_fund['s26_donors']['account_id'].values)*100),2)

In [ ]:
#Which season 25 donors churned in season 26?
churn_s26 = [a for a in annual_fund['s25_donors']['account_id'].values if a not in annual_fund['s26_donors']['account_id'].values]
#How many churned?
len(churn_s26)

In [ ]:
#Season 26 churn rate (based on specific donors):
churn_rate_s26 = round(len(churn_s26)/len(annual_fund['s25_donors']['account_id'].values)*100, 2)
churn_rate_s26

In [ ]:
#Season 26 churn rate (based on donor counts):
round(((len(annual_fund['s25_donors']['account_id'].values) - len(annual_fund['s26_donors']['account_id'].values))/len(annual_fund['s25_donors']['account_id'].values)*100),2)

In [ ]:
#Which season 24 donors churned in season 25?
churn_s25 = [a for a in annual_fund['s24_donors']['account_id'].values if a not in annual_fund['s25_donors']['account_id'].values]
#How many churned?
len(churn_s25)

In [ ]:
#Season 25 churn rate (based on specific donors):
churn_rate_s25 = round(len(churn_s25)/len(annual_fund['s24_donors']['account_id'].values)*100, 2)
churn_rate_s25

In [ ]:
#Season 25 churn rate (based on donor counts):
round(((len(annual_fund['s24_donors']['account_id'].values) - len(annual_fund['s25_donors']['account_id'].values))/len(annual_fund['s24_donors']['account_id'].values)*100),2)

In [ ]:
#Which season 24 donors churned in season 25?
churn_s24 = [a for a in annual_fund['s23_donors']['account_id'].values if a not in annual_fund['s24_donors']['account_id'].values]
#How many churned?
len(churn_s24)

In [ ]:
#Season 24 churn rate (based on specific donors):
churn_rate_s24 = round(len(churn_s24)/len(annual_fund['s23_donors']['account_id'].values)*100, 2)
churn_rate_s24

In [ ]:
#Season 24 churn rate (based on donor counts):
round(((len(annual_fund['s23_donors']['account_id'].values) - len(annual_fund['s24_donors']['account_id'].values))/len(annual_fund['s23_donors']['account_id'].values)*100),2)

First, defining retention as donor gave to the annual fund for two or more consecutive seasons

In [ ]:
#As DICTIONARY:
#stp_amt_list = ['stp_total_s27', 'stp_total_s26', 'stp_total_s25', 'stp_total_s24', 'stp_total_s23']
#donor_levels_by_season_dict_of_dfs = {}
#donor_levels_by_season['account_id'] = donations_no_outlier_amts['account_id']
#for col in stp_amt_list:
#    df_name = col[:3] + col[9:] + 'level'
#    donor_levels_by_season[df_name] = (donations_no_outlier_amts[col].apply(stp_level_generator))

In [ ]:
#Store a new dataframe with each iteration as a new dictionary value
#stp_amt_list = ['stp_total_s27', 'stp_total_s26', 'stp_total_s25', 'stp_total_s24', 'stp_total_s23']
#donor_levels_by_season_dict_of_dfs = {} #initialize empty dictionary

#for col in stp_amt_list:
#    new_df_name = col[:3] + col[9:] + 'level' 
#    donor_levels_by_season_dict_of_dfs[new_df_name] = donations_no_outlier_amts[col].apply(stp_level_generator)

# Left Off Here

Separate continuous and categorical features for exploration.

In [ ]:
cats['donation_record_type'].value_counts().plot.bar(title='Freq dist of Donation Record Type', figsize=[15,10])

In [ ]:
donations['fund'].value_counts().plot.bar(title='Freq dist of Fund', figsize=[15,10])

In [ ]:
cont_vars

Continuous Plots

In [ ]:
x = less_max['amount']
ax = sns.distplot(x)

In [ ]:
fig, ax = plt.subplots(len(cont_vars), figsize=(16,12))

for i, col_val in enumerate(cont_vars):
    try:
        sns.distplot(cont[col_val], hist=True, ax=ax[i])
        ax[i].set_title('Freq dist '+col_val, fontsize=10)
        ax[i].set_xlabel(col_val, fontsize=8)
        ax[i].set_ylabel('Count', fontsize=8)
    except RuntimeError as re:
        if str(re).startswith("Selected KDE bandwidth is 0. Cannot estimate density."):
            sns.distplot(cont[col_val], hist=True, ax=ax[i], kde_kws={'bw': 0.1})
        else:
            raise re
    
plt.show()

In [ ]:
cont_dollars = ['amount', 'pledge_amount', 'amount_matched', 'last_donation_amount','lifetime_donation_total_amount',
       'lifetime_single_ticket_purchase_total', 'lifetime_subscription_purchase_total', 'amount_paid', 'amount_donated_last_365_days',
       'amount_donated_last_fiscal_year', 'amount_donated_this_fiscal_year', 'group_sales_total', 'total_amount']

In [ ]:
fig, ax = plt.subplots(len(cont_dollars), figsize=(16,12))

for i, col_val in enumerate(cont_dollars):
    try:
        sns.distplot(less_max[col_val], hist=True, ax=ax[i])
        ax[i].set_title('Freq dist '+col_val, fontsize=10)
        ax[i].set_xlabel(col_val, fontsize=8)
        ax[i].set_ylabel('Count', fontsize=8)
    except RuntimeError as re:
        if str(re).startswith("Selected KDE bandwidth is 0. Cannot estimate density."):
            sns.distplot(less_max[col_val], hist=True, ax=ax[i], kde_kws={'bw': 0.1})
        else:
            raise re
    
plt.show()

In [ ]:
fig, ax = plt.subplots(len(cont_vars), figsize=(16,12))

for i, col_val in enumerate(cont_vars):
    try:
        sns.distplot(cont[col_val], hist=True, ax=ax[i])
        ax[i].set_title('Freq dist '+col_val, fontsize=10)
        ax[i].set_xlabel(col_val, fontsize=8)
        ax[i].set_ylabel('Count', fontsize=8)
    except RuntimeError as re:
        if str(re).startswith("Selected KDE bandwidth is 0. Cannot estimate density."):
            sns.distplot(cont[col_val], hist=True, ax=ax[i], kde_kws={'bw': 0.1})
        else:
            raise re
    
plt.show()